In [1]:
from peft import PeftModel, PeftConfig

In [2]:
!pip install bitsandbytes

In [3]:
checkpoint_model = "manojbaniya/roman-ecommerce-qna"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint_model)
model = AutoModelForCausalLM.from_pretrained(checkpoint_model, load_in_8bit=True, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` o

In [5]:
## inference test for try
prompt = "account kasari kholne"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


account kasari kholne ko lagi kisi tarika batao

### Response:
Account kasari kholne ko lagi kisi tarika batao


In [6]:
prompt = """Answer the question of Ecommerce Customer Support in Roman Nepali Language.

### Question:
{}

### Response:
{}"""

In [7]:
input = prompt.format("account kasari kholne", "")
input

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\naccount kasari kholne\n\n### Response:\n'

In [8]:
inputs = tokenizer(input, return_tensors="pt").to("cuda")
inputs["input_ids"]

tensor([[     2,   1261,    573,   2872,    576, 218666,  13653,  10214,    575,
           9783, 205184,  14944, 235265,    109,   6176,  10825, 235292,    108,
           9600,  13097,   1050,   4190,    561,    510,    109,   6176,  10567,
         235292,    108]], device='cuda:0')

In [9]:
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Answer the question of Ecommerce Customer Support in Roman Nepali Language.

### Question:
account kasari kholne

### Response:
account kholne ko lagi form fill garnu hos.


In [10]:
def generate_answer(question: str) -> str:
    input = prompt.format(question, "")
    inputs = tokenizer(input, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
generate_answer("delivery option ke cha hau")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\ndelivery option ke cha hau\n\n### Response:\ndelivery option ko lagi order customize garna milcha.'

In [12]:
generate_answer("subscription lina ke garne")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nsubscription lina ke garne\n\n### Response:\nsubscription ko lagi payment process garna sakyo'

### Inference with Unsloth

In [13]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [14]:
!pip install unsloth

In [15]:
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-k7j7cgre
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-k7j7cgre
  Resolved https://github.com/unslothai/unsloth.git to commit b4c48d9c5e78203909495bf9beaa29a5c9aeaeeb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.5-py3-none-any.whl size=176840 sha256=071239b8171727b8285ea42190816901b5943ce002a9630befc5a417b495370f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4hcb133b/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [16]:
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [17]:
from unsloth import FastLanguageModel
model_unsloth, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "manojbaniya/roman-ecommerce-qna", # MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model_unsloth) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.1.5 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [18]:
!pip install triton

In [19]:
inputs = tokenizer(
[
    prompt.format(
        "account kasari kholne", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model_unsloth.generate(**inputs, max_new_tokens = 100)

In [20]:
tokenizer.decode(outputs[0])

'<bos>Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\naccount kasari kholne\n\n### Response:\naccount kholne ko lagi form fill garnu hos.<eos>'

In [21]:
def chat_unsloth(question: str) -> str:
    question = prompt.format(question, "")
    inputs = tokenizer(question, return_tensors="pt").to("cuda")
    outputs = model_unsloth.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [22]:
chat_unsloth("delivery option ke cha hau")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\ndelivery option ke cha hau\n\n### Response:\ndelivery option ko lagi order customize garna milcha.'

In [23]:
chat_unsloth("reward k k cha")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nreward k k cha\n\n### Response:\nreward ko lagi points redeem garna milcha'

In [24]:
chat_unsloth("What is AI")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nWhat is AI\n\n### Response:\nAI is a technology that helps machines understand and respond to human language.'

In [25]:
chat_unsloth("timro name ke ho")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\ntimro name ke ho\n\n### Response:\ntimro name ko lagi sanchaari huncha'

### Trying the original model

In [27]:
question = "account kasari kholne"
input = prompt.format(question, "")
inputs = tokenizer(input, return_tensors="pt").to("cuda")
outputs = model_unsloth.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Answer the question of Ecommerce Customer Support in Roman Nepali Language.

### Question:
account kasari kholne

### Response:
account kholne ko lagi form fill garnu hos.


### different

In [28]:
# load from unsloth model gemma original
checkpoint_model = "unsloth/gemma-2-2b-bnb-4bit"

from unsloth import FastLanguageModel

model_gemma, tokenizer = FastLanguageModel.from_pretrained(
        model_name = checkpoint_model, # MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model_gemma)

==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layerno

In [29]:
inputs = tokenizer(
[
    prompt.format(
        "account kasari kholne", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model_gemma.generate(**inputs, max_new_tokens = 100)

In [30]:
tokenizer.decode(outputs[0])

'<bos>Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\naccount kasari kholne\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open\n\n### Question:\naccount open\n\n### Response:\nAccount open'

In [48]:
prompt = """Answer the question of Ecommerce Customer Support in Roman Nepali Language.

### Question:
{}

### Response:
{}"""

In [58]:
def chat_with_gemma(question: str) -> str:
    # question = prompt.format(question, "")
    inputs = tokenizer(question, return_tensors="pt").to("cuda")
    outputs = model_gemma.generate(**inputs, max_new_tokens=40)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
prompt.format("", "")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\n\n\n### Response:\n'

In [41]:
chat_with_gemma("delivery option ke cha hau")

'delivery option ke cha hau\n\n[User 0001]\n\nI am trying to find out what the delivery option is for the 2007 328i. I have been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told that it is the same as the 335i. I have also been told'

In [33]:
chat_unsloth("delivery option ke cha hau")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\ndelivery option ke cha hau\n\n### Response:\ndelivery option ko lagi order customize garna milcha.'

In [37]:
chat_with_gemma("The sky is") # for next 200 token

'The sky is the limit for the 2019-2020 season of the <strong>National League</strong>. The <strong>National League</strong> is a professional baseball league in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues in the United States. The <strong>National League</strong> is the oldest of the two major professional baseball leagues i

In [42]:
chat_with_gemma("free reward available xa")

'free reward available xa2000\n\n[User 0001]\n\nHi,\n\nI have a free reward available for a xa2000.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and would like to give it away.\n\nI have a 2000 that I am not using and'

In [47]:
chat_unsloth("free reward available xa vane vanidinu")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nfree reward available xa vane vanidinu\n\n### Response:\nfree reward available ko lagi website ma check garnus.'

In [51]:
chat_with_gemma("free reward available xa vane vanidinu hai")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nfree reward available xa vane vanidinu hai\n\n### Response:\nThank you for your question. We are sorry to hear that you are having trouble with your order. We will be happy to help you. Please contact us at support@ecommerce.com. Thank you for your patience.'

In [53]:
chat_with_gemma("What is AI")

'What is AI?\n\nArtificial Intelligence (AI) is a branch of computer science that deals with the creation of intelligent machines that can perform tasks that typically require human intelligence. AI is a broad field that encompasses many different types of machines, from simple robots to complex artificial neural networks.\n\nWhat is the difference between AI and machine learning?\n\nAI is a broad field that encompasses many different types of machines, from simple robots to complex artificial neural networks. Machine learning is a subset of AI that focuses on the development of algorithms that can learn from data and make decisions based on that data.\n\nWhat is the difference between AI and machine learning?\n\nAI is a broad field that encompasses many different types of machines, from simple robots to complex artificial neural networks. Machine learning is a subset of AI that focuses on the development of algorithms that can learn from data and make decisions based on that data.\n\n

In [54]:
chat_with_gemma("What happens if we step on ice answer in just a few sentence")

'What happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if we step on ice answer in just a few sentence.\n\nWhat happens if'

In [55]:
chat_unsloth("What happens if we step on ice answer in just a few sentence")

'Answer the question of Ecommerce Customer Support in Roman Nepali Language.\n\n### Question:\nWhat happens if we step on ice answer in just a few sentence\n\n### Response:\nStep on ice can cause injury.'

In [59]:
chat_with_gemma("Cat sat on a ")

'Cat sat on a 100-foot-high cliff. She was 100 feet above the ground. She jumped off the cliff. She fell 100 feet. How long did it take her'

In [62]:
chat_with_gemma("President of USA has")

'President of USA has been in the news for his controversial statements and actions. He has been in the news for his controversial statements and actions. He has been in the news for his controversial statements and actions. He has been'

In [67]:
chat_with_gemma("where does Lionel Messi live?")

'where does Lionel Messi live?\n\nLionel Messi is a professional footballer who plays for the Argentina national team and the Barcelona club. He is one of the best players in the world and is known for his skills and abilities on the field'

In [69]:
chat_with_gemma("Why has Nepal never been colonized?")

'Why has Nepal never been colonized?\n\nWhy has Nepal never been colonized?\n\nWhy has Nepal never been colonized?\n\nWhy has Nepal never been colonized?\n\nWhy has Nepal never been colonized?\n\nWhy has Nepal never been colonized?'

In [70]:
chat_with_gemma("Nepal has never been colonized because")

'Nepal has never been colonized because it is a landlocked country. It is a landlocked country because it is surrounded by India on three sides.\n\nNepal is a landlocked country because it is surrounded by India on three sides.'

In [71]:
chat_with_gemma("what is the capital city of Nepal?")

'what is the capital city of Nepal?\n\nWhat is the capital city of Nepal?\n\nWhat is the capital city of Nepal?\n\nWhat is the capital city of Nepal?\n\nWhat is the capital city of Nepal?\n\nWhat is the'

In [72]:
chat_with_gemma("The capital city of Nepal is ")

'The capital city of Nepal is <strong>Kathmandu</strong>. It is the largest city in Nepal and the country’s economic, cultural, and political center. It is located in the central part of the country, at an elevation of'